In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model


In [2]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_k(x,k):
    return 1 / (1 + k*np.exp(-z))

In [3]:
def relu(x):
    return np.log(1+np.exp(x))

In [ ]:
#out_layer_act
def drelu(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

In [4]:
def loss(out,Y):
        loss = (-1)*(np.sum(np.multiply(np.log(out), Y) + np.multiply((1 - Y), np.log(1 - out))))/(Y.shape[1])
        #print(loss)
        return loss

In [49]:
#testimg relu and sigmoid
#relu(5)
#relu(-3)
#relu(0)

#sigmoid(5)
#sigmoid(0)
#sigmoid(-4)

In [7]:
df_proc = pd.read_csv("preprocessed.csv") 

In [8]:
df_proc.columns=["sl_no","id","thickness", "cellsize", "cellshape", "madhesion", "secellsize", "barenuclei", "chromatin", "nucleoli", "mitosis","Class"]

In [9]:
Y = df_proc.filter(['Class'], axis=1)
X = df_proc.drop(['sl_no','id','Class'],axis=1)

In [10]:
X.shape
X,Y=X.to_numpy(),Y.to_numpy()
X,Y=X.T,Y.reshape(1, Y.shape[0])

In [11]:
print(X.shape,Y.shape)

#split data to X train Xtest Y train Ytest

(9, 699) (1, 699)


In [12]:
#taking all 9 layers as input
in_layer_no=X.shape[0] #no of attributes
hid_layer_no=int(1.5*in_layer_no)
out_layer_no=1

#initial weights
wh = np.random.randn(hid_layer_no,in_layer_no) * 0.01
bh = np.zeros(shape=(hid_layer_no, 1))
w_out = np.random.randn(out_layer_no,hid_layer_no) * 0.01
b_out = np.zeros(shape=(out_layer_no, 1))

initial_weights=[wh,bh,w_out,b_out]

In [276]:
print(in_layer_no)
print(hid_layer_no)
print(out_layer_no)
wh.shape

9
13
1


(13, 9)

In [28]:
def ann(X,Y,learning_rate,initial_weights):
    wh=initial_weights[0]
    bh=initial_weights[1]
    w_out=initial_weights[2]
    b_out=initial_weights[3]
    dwh_old=0
    dw_out_old=0
    for i in range(0,100000):
        #forward propogation
        #input to hidden layer = dot product(X,wh) + bh
        hid_layer_input = np.dot(wh,X) + bh
        hid_layer_act = relu(hid_layer_input)
        
        # Final output layer prediction
        out_layer_input = np.dot(w_out,hid_layer_act) + b_out
        out_layer_act = sigmoid(out_layer_input)
        lo=loss(out_layer_act,Y)
        if(i%1000==0):
            print(i,lo)
        '''dZ2 = out_layer_act - Y
        dW2 = (1 /X.shape[1]) * np.dot(dZ2, hid_layer_act.T)
        db2 = (1 / X.shape[1]) * np.sum(dZ2, axis=1, keepdims=True)
        dZ1 = np.multiply(np.dot(w_out.T, dZ2), 1 - np.power(hid_layer_act, 2))
        dW1 = (1 / X.shape[1]) * np.dot(dZ1, X.T)
        db1 = (1 / X.shape[1]) * np.sum(dZ1, axis=1, keepdims=True)
        
        wh = wh - learning_rate * dW1
        w_out = w_out - learning_rate * dW2
        bh = bh - learning_rate * db1
        b_out = b_out - learning_rate * db2
        '''
        #backward propogation output layer    
        t_o = out_layer_act - Y
        sigmak = t_o * sigmoid(out_layer_input)*(1-sigmoid(out_layer_input))   
        dLoss_W2 = (1/hid_layer_act.shape[1]) * np.dot(sigmak,hid_layer_act.T)
        dLoss_b2 = (1/hid_layer_act.shape[1]) * np.dot(sigmak, np.ones([sigmak.shape[1],1])) 
          
        #backward propogation input layer
        dLoss_A1 = np.dot(w_out.T,sigmak)
        dLoss_Z1 = dLoss_A1 * drelu(hid_layer_input)        
        dLoss_A0 = np.dot(wh.T,dLoss_Z1)
        dLoss_W1 = 1/X.shape[1] * np.dot(dLoss_Z1,X.T)
        dLoss_b1 = 1/X.shape[1] * np.dot(dLoss_Z1, np.ones([dLoss_Z1.shape[1],1]))  
        
        wh = wh - learning_rate * dLoss_W1
        w_out = w_out - learning_rate * dLoss_W2
        bh = bh - learning_rate * dLoss_b1
        b_out = b_out - learning_rate * dLoss_b2
    return [wh,bh,w_out,b_out]
    ''' 
        #backpropogation
        dout_layer_act = (out_layer_act - Y) / (out_layer_act * (1 - out_layer_act))
        dZ2 = np.multiply(dout_layer_act, out_layer_act * (1 - out_layer_act))
        dw_out = np.dot(dZ2, hid_layer_act.T)
        #dw_out=dw_out+0.5*dw_out_old
        #dw_out_old=dw_out
        db_out = np.sum(dZ2, axis=1, keepdims=True)

        dhid_layer_act = np.dot(w_out.T, dZ2)
        dZ1 = np.multiply(dhid_layer_act, dhid_layer_act * (1 - dhid_layer_act))
        dwh = np.dot(dZ1, X.T)
        #dwh=dwh+0.2*dwh_old
        #dwh_old=dwh
        dbh = np.sum(dZ1, axis=1, keepdims=True)
        
        wh = wh - learning_rate * dwh
        w_out = w_out - learning_rate * dw_out
        bh = bh - learning_rate * dbh
        b_out = b_out - learning_rate * db_out
    '''
        
        

In [29]:
#here replace X with X train and Y with Y train
final_weights=ann(X,Y,0.05,initial_weights)

0 0.6998187948987924
1000 0.6377822950995017
2000 0.6215885807945587
3000 0.5423958611897002
4000 0.18946447788995283
5000 0.13442865781959734
6000 0.11786630856428194
7000 0.1098364973919441
8000 0.10503059148697583
9000 0.10184046164545175
10000 0.09961649967791852
11000 0.09795551890258615
12000 0.09664494375198177
13000 0.09564070525165196
14000 0.09483033610268977
15000 0.09419691926507162
16000 0.09369897434410251
17000 0.09321480540548353
18000 0.09281324563995698
19000 0.09253856960964775
20000 0.09225075271428093
21000 0.09188651401502948
22000 0.09165593095456272
23000 0.0914879316766189
24000 0.09138470112713933
25000 0.09127740272246132
26000 0.09118781572941913
27000 0.09111187727778429
28000 0.0910435129800273
29000 0.09097496641396154
30000 0.09093066903963969
31000 0.0908979979318542
32000 0.09087444311791383
33000 0.09085544691235385
34000 0.09083881046897463
35000 0.09082332454456817
36000 0.09081046531741925
37000 0.09079245116856845
38000 0.09077873364685288
39000 0

In [33]:
#final backpropogation to test training data 
hid_layer_input = np.dot(final_weights[0],X) + final_weights[1]
hid_layer_act = relu(hid_layer_input)
out_layer_input = np.dot(final_weights[2],hid_layer_act) + final_weights[3]
out_layer_act = sigmoid(out_layer_input)
predictions = out_layer_act > 0.5
predictions=predictions.astype(int)  

#here replace the X with X test

In [35]:
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')
#replace Y with Y test

Accuracy: 97%
